This notebook ingests NCA geotifs for the 'public website', applies colours to them based on examples provided alongside the data, and then generates static map tilesets, using the xyz tile schema. These tilesets are then uploaded to a dedicated S3 bucket.
Resources:
link to matplotlib colormaps - https://matplotlib.org/stable/gallery/color/colormap_reference.html

In [49]:
import os
import numpy as np
import math
import rasterio
from rasterio.plot import reshape_as_image
from matplotlib.colors import Normalize, ListedColormap

from matplotlib.cm import get_cmap
import gdal2tiles
#import folium
#from IPython.display import display

In [50]:
"""
Not gunna lie... chatGPT figured out how to calculate the minimum zoom level per geotif.
"""
def calculate_min_zoom_level(dataset):
    # Earth's radius in metres
    earth_radius = 6378137

    # Calculate pixel size in meters at the equator
    pixel_size_x = (dataset.bounds.right - dataset.bounds.left) / dataset.width
    pixel_size_y = (dataset.bounds.top - dataset.bounds.bottom) / dataset.height
    pixel_size = max(pixel_size_x, pixel_size_y)

    # Calculate ground resolution at the equator
    equator_circumference = 2 * math.pi * earth_radius
    ground_res_at_equator = pixel_size / equator_circumference * 360

    # Find the closest zoom level where the ground resolution is higher than the tile resolution
    min_zoom_level = 0
    for zoom_level in range(21):  # Web maps typically use zoom levels 0 to 20
        tile_ground_res = 360 / (256 * 2**zoom_level)
        if tile_ground_res <= ground_res_at_equator:
            min_zoom_level = zoom_level
            break

    return min_zoom_level


In [51]:

def generate_coloured_tif(data, colour_map_name, original_nodata, discrete):
    cmap = get_cmap(colour_map_name)
    if discrete:
        # Define colors to avoid white for the first class in discrete rasters
        colors = [cmap(0.5), cmap(1.0)]  # Adjust as needed
        new_cmap = ListedColormap(colors)
        norm = Normalize(vmin=np.nanmin(data), vmax=np.nanmax(data))
        use_cmap = new_cmap  # Use the new discrete colormap
    else:
        # Use continuous colormap
        norm = Normalize(vmin=np.nanmin(data), vmax=np.nanmax(data))
        use_cmap = cmap  # Use the original continuous colormap

    norm_data = norm(data)
    norm_data = np.nan_to_num(norm_data, nan=original_nodata)
    colored_data = use_cmap(norm_data)[:, :, :3]  # Apply the selected colormap
    return (colored_data * 255).astype(np.uint8)



def process_geotiff(input_geotiff_path, output_directory, colour_map_name, discrete):
    try:
        with rasterio.open(input_geotiff_path) as src:
            tif_data = src.read(1, masked=True).astype('float32')
            original_nodata = src.nodata

            # Generate the colored GeoTIFF image
            coloured_image = generate_coloured_tif(tif_data, colour_map_name, original_nodata, discrete)

            # Define metadata for the colored GeoTIFF
            meta = src.meta.copy()
            meta.update(count=3, dtype='uint8', nodata=0)  # 3 channels, uint8, no nodata

            # Save the colored GeoTIFF locally
            file_name = os.path.splitext(os.path.basename(input_geotiff_path))[0]
            coloured_geotiff_path = os.path.join(output_directory, f"{file_name}-coloured.tif")
            with rasterio.open(coloured_geotiff_path, 'w', **meta) as dst:
                dst.write(coloured_image.transpose(2, 0, 1))  # Reorder to match band order in rasterio

            return {
                'message': 'Colored GeoTIFF created successfully.',
                'coloured_geotiff_path': coloured_geotiff_path
            }
    except Exception as e:
        return {
            'error': str(e)
        }

def create_map_tiles(geotiff_path, output_directory):
    with rasterio.open(geotiff_path) as dataset:
        min_zoom = calculate_min_zoom_level(dataset)
        print(f"Min zoom level calculated: {min_zoom}")

    tiles_output_directory = os.path.join(output_directory, 'map_tiles')
    os.makedirs(tiles_output_directory, exist_ok=True)

    try:
        gdal2tiles.generate_tiles(
            geotiff_path,
            tiles_output_directory,
            zoom=[min_zoom, 18],  # Adjust max zoom as needed
            tmscompatible=True,
            #srcnodata=0,
            #nodata=0,
            verbose=False
        )
        print("Tiles generated successfully.")
    except Exception as e:
        print("An error occurred during tile generation:", e)

    return tiles_output_directory, min_zoom

The below cell has all the NCA tifs listed in order, and the colormaps for eah one listed in matching order afterwards. The transparency issue for the map tiles should be the only thing that needs to be bug fixed.Note - change the minimum zoom for the maptiles to be set dynamically based on function that calcualtes min zoom. Set max zoom at 18 or so - higher levels will take a long time to generate.

In [52]:
input_tifs = [
    'birds/YQ_Allbirds_gam.tif',
    'birds/YQ_Grassbirds_gam.tif',
    'birds/YQ_Woodbirds_gam.tif',
    'groundcover/YQ-2022-GC10.tif',
    'groundcover/YQ-2022-MEANGC10.tif',
    'invertebrates/YQdet_map.tif',
    'plants/YQ_AllNativePlants_gam.tif',
    'plants/YQ_AllPlants_gam.tif',
    'plants/YQ_NativeGroundLayer_gam.tif',
    'plants/YQ_NativeShrubs_gam.tif',
    'trees-shade-shelter/YQ-NCI5-Proximity.tif'
]

#geotifs that need discrete colors instead of continuous:
input_tifs_discrete = [
    'trees-shade-shelter/YQ-shade-footprint-onoffsitecontribution-production-only.tif',
    'trees-shade-shelter/YQ-summer-shelter-onoffcontrib-production-zone.tif',
    'trees-shade-shelter/YQ-winter-shelter-onoffcontrib-production-zone.tif'
]

#the cmaps are matplotlib in same order as the above geotifs to make sure the right colours are applied.
cmap_for_input_rasters=[
    'Blues',    
    'Blues',
    'Blues',
    'RdYlGn',
    'RdYlGn',
    'YlOrBr',
    'Purples',
    'Purples',
    'Purples',
    'Purples',
    'Blues'
]

# matplotlib colormaps for discrete geotifs:
cmap_for_input_rasters_discrete=[
    'Greens',
    'Reds', 
    'Blues' 
]

Loop through the geotif files in the root directory for the Orana farm (sorry for the hard coded filenames) and apply discrete or continuous colormaps to them based on Danny's supplied map designs.

In [53]:
base_input_directory = 'data/Orana/Orana Rasters/'
base_output_directory = 'data/Orana/output_rasters/'

# Ensure the base output directory exists
os.makedirs(base_output_directory, exist_ok=True)

# Processing each GeoTIFF with its corresponding colormap
for input_tif, cmap_name in zip(input_tifs, cmap_for_input_rasters):
    input_geotiff_path = os.path.join(base_input_directory, input_tif)
    file_name_without_ext = os.path.splitext(os.path.basename(input_tif))[0]
    output_directory_for_colored_tif = os.path.join(base_output_directory, os.path.dirname(input_tif))
    output_directory_for_tiles = os.path.join(output_directory_for_colored_tif, f"{file_name_without_ext}_maptiles")

    # Ensure output directories for colored tif and tiles exist
    os.makedirs(output_directory_for_colored_tif, exist_ok=True)
    os.makedirs(output_directory_for_tiles, exist_ok=True)

    # Process each GeoTIFF file
    processing_result = process_geotiff(input_geotiff_path, output_directory_for_colored_tif, cmap_name, discrete=True)

    if 'coloured_geotiff_path' in processing_result:
        tiles_directory, min_zoom = create_map_tiles(processing_result['coloured_geotiff_path'], output_directory_for_tiles)
        print(f"Tiles directory for {input_tif}: {tiles_directory}")
    else:
        print(f"Error in processing GeoTIFF {input_tif}: {processing_result.get('error', 'Unknown Error')}")


for input_tif, cmap_name in zip(input_tifs_discrete, cmap_for_input_rasters_discrete):
    input_geotiff_path = os.path.join(base_input_directory, input_tif)
    file_name_without_ext = os.path.splitext(os.path.basename(input_tif))[0]
    output_directory_for_colored_tif = os.path.join(base_output_directory, os.path.dirname(input_tif))
    output_directory_for_tiles = os.path.join(output_directory_for_colored_tif, f"{file_name_without_ext}_maptiles")

    # Ensure output directories for colored tif and tiles exist
    os.makedirs(output_directory_for_colored_tif, exist_ok=True)
    os.makedirs(output_directory_for_tiles, exist_ok=True)

    # Process each GeoTIFF file
    processing_result = process_geotiff(input_geotiff_path, output_directory_for_colored_tif, cmap_name, discrete=True)

    if 'coloured_geotiff_path' in processing_result:
        tiles_directory, min_zoom = create_map_tiles(processing_result['coloured_geotiff_path'], output_directory_for_tiles)
        print(f"Tiles directory for {input_tif}: {tiles_directory}")
    else:
        print(f"Error in processing GeoTIFF {input_tif}: {processing_result.get('error', 'Unknown Error')}")

/tmp/ipykernel_763/3679397071.py:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = get_cmap(colour_map_name)


Min zoom level calculated: 14
Generating Base Tiles:
0...10...20...30...40...50...60